# 繁體中文詞性標註系統 by CKIP transformers
#### 詞性標記使用辭典：中研院資訊所、語言所詞庫小組所編技術報告第 95-02/98-04 ：https://asbc.iis.sinica.edu.tw/
#### Cite：此開源碼來自中研院CKIP Lab：https://github.com/ckiplab/ckip-transformers
#### Modified by：國立清華大學電機所  賴彥霖


### 登入Google 帳號

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 安裝套件

In [ ]:
!pip install ckip_transformers
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.8 MB/s eta 0:00:00


#### 導入套件

In [ ]:
import os
import numpy as np
import pandas as pd
import sys
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
sys.path.append('/content/drive/MyDrive')
from tool import pos

### 更改filename為欲分析的逐字稿檔名：下方將列出檔案中逐字稿內容
##### 注意事項1：檔名外需有引號 ' ' 包覆，如 '111050001_task2.xlsx'
##### 注意事項2：檔名需有檔案類型，如'111050008_task2.xlsx'，不可為'111050008_task2'，只接受excel檔。

### 直接將統計完數據儲存至雲端硬碟

In [ ]:
filename = '111050005_task1.xlsx'
pos(filename)

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.35s/it]


'Your file has been saved to /content/drive/MyDrive/詞性分析逐字稿/111050005_task1.xlsx'

### (舊版)下方可顯示每句詞性標註結果


In [ ]:
filename = '111050064_task2.xlsx'
dirpath = '/content/drive/MyDrive/逐字稿/所有逐字稿excel'
file_path = os.path.join(dirpath, filename)
df = pd.read_excel(file_path, usecols=["Content"])
print(df)

pos_EtoC = { "A":"形容詞" ,"Caa":"X", "Cab":"X", "Cba":"X", "Cbb":"X", "D":"副詞", "Da":"X", "Dfa":"副詞", "Dfb":"副詞",
        "Di":"X", "Dk":"X", "DM":"X", "I":"X", "Na":"名詞", "Nb":"名詞", "Nc":"名詞", "Ncd":"名詞", "Nd":"X", # "Nd":"時間詞",
        "Nep":"代名詞", "Neqa":"形容詞", "Neqb":"X", "Nes":"X", "Neu":"X", # "Neu":"數詞定詞",
        "Nf":"X", "Ng":"X", "Nh":"代名詞", "Nv":"X", "P":"X", # "P":"介詞",
        "T":"X", # "T":"語助詞",
        "VA":"動詞", "VAC":"動詞", "VB":"動詞", "VC":"動詞", "VCL":"X", "VD":"動詞", "VF":"動詞", "VE":"動詞", "VG":"動詞", "VH":"形容詞", "VHC":"動詞", "VI":"動詞",
        "VJ":"動詞", "VK":"動詞", "VL":"動詞", "V_2":"有", "DE":"的之得地", "SHI":"是","FW":"X",
        "COLONCATEGORY":"X", "COMMACATEGORY":"X", "DASHCATEGORY":"X", "DOTCATEGORY":"X", "ETCCATEGORY":"X", "EXCLAMATIONCATEGORY":"X", "PARENTHESISCATEGORY":"X",
        "PAUSECATEGORY":"X", "PERIODCATEGORY":"X", "QUESTIONCATEGORY":"X", "SEMICOLONCATEGORY":"X", "SPCHANGECATEGORY":"X", "WHITESPACE":"X", "X":"X"}
# Initialize drivers
ws_driver  = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")
ner_driver = CkipNerChunker(model="bert-base")
# Save transcripts to list
text = []
for i in range(len(df)):
   text.append(df["Content"].iloc[i])
# Enable sentence segmentation
ws  = ws_driver(text, use_delim=True)
ner = ner_driver(text, use_delim=True)
pos = pos_driver(ws, delim_set='\n\t')
# Pack word segmentation and part-of-speech results
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
   assert len(sentence_ws) == len(sentence_pos)
   res = []
   for word_ws, word_pos in zip(sentence_ws, sentence_pos):
      if word_pos == "X":
        res.append(f"{word_ws}")
      else:
        res.append(f"{word_ws}({word_pos})")
   return "\u3000".join(res)
# Show results
exclude = ["(", ")", "（", "台", "）", "，", "。", "...", "..", ".", " ", "?", "？", "!", "、"]
total_counts = [0]*8
print()
for sentence, sentence_ws, sentence_pos in zip(text, ws, pos): #會迭代每句的分詞、詞性(英文)
  print()
  print(sentence)
  sen_counts = [0]*8
  pos_ch = []
  for i in range(len(sentence_pos)):
    if sentence_ws[i]=="有":
      if sentence_ws[i-1] == "還":
        sentence_pos[i] = "X"
      elif pos_EtoC[sentence_pos[i+1]] == "動詞":
        sentence_pos[i] = "X"
      else:
        sentence_pos[i] = "VA"

    if sentence_ws[i] == "台":
      sentence_pos[i] = "X"
    pos_ch.append(pos_EtoC[sentence_pos[i]])
  new_sentence_ws = []                  # 新的分詞和詞性列表
  new_sentence_pos = []
  i = 0
  while i < len(sentence_ws):              # 遍歷詞和詞性，進行合併操作
      if pos_ch[i] == '形容詞' and i + 1 < len(sentence_ws) and pos_ch[i + 1] == '的之得地':
          new_sentence_ws.append(sentence_ws[i] + sentence_ws[i + 1])   # 合併形容詞和的之得地
          new_sentence_pos.append('形容詞')
          i += 2  # 跳過下一個詞
      else:
          new_sentence_ws.append(sentence_ws[i])  # 否則正常添加
          new_sentence_pos.append(pos_ch[i])
          i += 1
  for i in range(len(new_sentence_pos)):
    # if new_sentence_pos[i] not in exclude:  #不計算標點符號到字數統計量
    if new_sentence_ws[i] not in exclude:      #不計算台語標註的(台)為字數統計量
      sen_counts[0] += len(new_sentence_ws[i])
      sen_counts[1] += 1
      if new_sentence_pos[i] == "名詞":
        sen_counts[2] += 1
      elif new_sentence_pos[i] == "代名詞":
        sen_counts[3] += 1
      elif new_sentence_pos[i] == "動詞":
        sen_counts[4] += 1
      elif new_sentence_pos[i] == "形容詞":
        sen_counts[5] += 1
      elif new_sentence_pos[i] == "副詞":
        sen_counts[6] += 1
      else:
        continue
  print(pack_ws_pos_sentece(new_sentence_ws, new_sentence_pos))
  print("{:=^50s}".format(""))
  sentence_variables = [("@字數統計：", sen_counts[0]), ("@詞彙量：  ", sen_counts[1]), ("名詞：     ",sen_counts[2]," 比例：","{:.2%}".format(sen_counts[2]/sen_counts[1])),
   ("代名詞：   ",sen_counts[3]," 比例：","{:.2%}".format(sen_counts[3]/sen_counts[1])),  ("動詞：     ",sen_counts[4]," 比例：","{:.2%}".format(sen_counts[4]/sen_counts[1])),
    ("形容詞：   ",sen_counts[5]," 比例：","{:.2%}".format(sen_counts[5]/sen_counts[1])), ("副詞：     ",sen_counts[6]," 比例：","{:.2%}".format(sen_counts[6]/sen_counts[1]))]
  for label, variable, *ratio in sentence_variables:
    print(label, variable, *ratio)
  for i in range(len(total_counts)):
      total_counts[i] += sen_counts[i]
  print()
  total_counts[7] += 1
print()
print("【總統計量】")
total_variables = [("總字數：", total_counts[0]), ("總詞彙：", total_counts[1]), ("總句數：", total_counts[7]), ("名詞：   ", total_counts[2]," 比例：","{:.2%}".format(total_counts[2]/total_counts[1])),("代名詞： ", total_counts[3]," 比例：","{:.2%}".format(total_counts[3]/total_counts[1])),
          ("動詞：   ", total_counts[4]," 比例：","{:.2%}".format(total_counts[4]/total_counts[1])), ("形容詞： ", total_counts[5]," 比例：","{:.2%}".format(total_counts[5]/total_counts[1])), ("副詞：   ", total_counts[6]," 比例：","{:.2%}".format(total_counts[6]/total_counts[1]))]
for label, variable, *totalratio in total_variables:
    print(label, variable, *totalratio)

        Content
0       這拿東西很危險
1  好像這個是東西.掉下來了
2         就這樣而已
3       幾個人喔三個人


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]



這拿東西很危險
這(代名詞)　拿(動詞)　東西(名詞)　很(副詞)　危險(形容詞)
@字數統計： 7
@詞彙量：   5
名詞：      1  比例： 20.00%
代名詞：    1  比例： 20.00%
動詞：      1  比例： 20.00%
形容詞：    1  比例： 20.00%
副詞：      1  比例： 20.00%


好像這個是東西.掉下來了
好像(副詞)　這(代名詞)　個　是(是)　東西(名詞)　.　掉下來(動詞)　了
@字數統計： 11
@詞彙量：   7
名詞：      1  比例： 14.29%
代名詞：    1  比例： 14.29%
動詞：      1  比例： 14.29%
形容詞：    0  比例： 0.00%
副詞：      1  比例： 14.29%


就這樣而已
就(副詞)　這樣(形容詞)　而已
@字數統計： 5
@詞彙量：   3
名詞：      0  比例： 0.00%
代名詞：    0  比例： 0.00%
動詞：      0  比例： 0.00%
形容詞：    1  比例： 33.33%
副詞：      1  比例： 33.33%


幾個人喔三個人
幾　個　人(名詞)　喔　三　個　人(名詞)
@字數統計： 7
@詞彙量：   7
名詞：      2  比例： 28.57%
代名詞：    0  比例： 0.00%
動詞：      0  比例： 0.00%
形容詞：    0  比例： 0.00%
副詞：      0  比例： 0.00%


【總統計量】
總字數： 30
總詞彙： 22
總句數： 4
名詞：    4  比例： 18.18%
代名詞：  2  比例： 9.09%
動詞：    2  比例： 9.09%
形容詞：  2  比例： 9.09%
副詞：    3  比例： 13.64%
